# 4.1 인스타그램 크롤링 

### 4.1.2 인스타그램 검색 결과 URL을 만들어서 접속하기

In [1]:
# 예제 4-1 인스타그램 검색결과 URL을 만드는 함수
def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + word
    return url
    

In [2]:
# 예제 4-2 selenium으로 URL 접속하기 - 1
from selenium import webdriver

driver = webdriver.Chrome()   

In [3]:
# 예제 4-2 selenium으로 URL 접속하기 - 2
import time
from selenium.webdriver.common.by import By

# 인스타그램 접속하기
driver.get('https://www.instagram.com/')
time.sleep(2)

In [7]:
# 예제 4-2 selenium으로 URL 접속하기 - 3

# HTML 틀 바뀌어서 찾아서 변경함.
# 인스타계정으로 로그인하기
email = '01041303038'   ### 계정 정보 수정 필요
password = 'moonwalk486!' ### 비번 정보 수정 필요

# 인스타그램 로그인 버튼 클릭
driver.find_element(By.CLASS_NAME, '_aa4b ').click()
time.sleep(2)

# 이메일 입력란 찾아서 값 입력
email_input = driver.find_element(By.CLASS_NAME, '_aa4b')
email_input.send_keys(email)

# 비밀번호 입력란 찾아서 값 입력
password_input = driver.find_elements(By.CLASS_NAME, '_aa4b')[1] 
password_input.send_keys(password)

# 로그인 버튼 클릭
login_button = driver.find_element(By.CSS_SELECTOR, 'button[type="submit"]')
login_button.click()

# 일부 웹사이트에서는 로그인 버튼의 클릭 후 잠시 대기가 필요할 수 있습니다.
time.sleep(2)  # 예시로 2초 대기


In [11]:
# 예제 4-2 selenium으로 URL 접속하기 - 4
word = "김채원"
url = insta_searching(word)
driver.get(url)

### 4.1.3 첫 번째 게시글 열기 

In [16]:
# 예제 4-3 HTML에서 첫번째 게시글 찾아 클릭하기

# 내용 일부 수정
def select_first(driver):
    first = driver.find_element(By.CSS_SELECTOR, "div._aagw")
    first.click()
    time.sleep(5)
    
select_first(driver)

### 4.1.4 게시글 정보 가져오기 

In [17]:
# 예제 4-4 게시글 정보 가져오기
import re
from bs4 import BeautifulSoup
import unicodedata

def get_content(driver):
    # ① 현재 페이지 html 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # ② 본문 내용 가져오기
    try:
        content = soup.select('div._a9zs')[0].text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    
    # ③ 본문 내용에서 해시태그 가져오기(정규식 활용)
    tags = re.findall(r'#[^Ws#,\\]+', content)
   
    # ④ 작성일자 정보 가져오기
    # time class 안에 datetime이라는 항목(년월일시분초 있음)에서 10개 string만 보겠다라는 뜻
    date = soup.select('time._a9ze')[0]['datetime'][:10] 

    # ⑤ 좋아요 수 가져오기
    try:
        like = soup.select('span.html-span')[2].text
    except:
        like = 0
        
    # ⑥ 위치정보 가져오기
    try: 
        place = soup.select('div._aaqm')[0].text
        place = unicodedata.normalize('NFC', place)      # 신규 추가. 2020.08.20
    except:
        place = ''
    # ⑦ 수집한 정보 저장하기
    data = [content, date, like, place, tags]
    return data
 
get_content(driver)

['03/03 채원이 인스타 게시물❤️너무 이쁘다아….🫢❤️#kimchaewon #chaewon #lesserafim #fearnot #김채원 #채원 #러블리더 #아기치타 #채채 #푸푸 #쌈무 #쌈아치 #쌈무요정 #모닝채원 #김도독 #핌둥이 #피어나 #르세라핌 #キムチェウォン #ルセラフィム',
 '2024-03-03',
 '1247',
 '',
 ['#kimchaewon ',
  '#chaewon ',
  '#le',
  '#fearnot ',
  '#김채원 ',
  '#채원 ',
  '#러블리더 ',
  '#아기치타 ',
  '#채채 ',
  '#푸푸 ',
  '#쌈무 ',
  '#쌈아치 ',
  '#쌈무요정 ',
  '#모닝채원 ',
  '#김도독 ',
  '#핌둥이 ',
  '#피어나 ',
  '#르세라핌 ',
  '#キムチェウォン ',
  '#ルセラフィム']]

### 4.1.5 다음 게시글 열기

In [18]:
# 예제 4-5 다음 게시글 열기
def move_next(driver):
    right = driver.find_element(By.CSS_SELECTOR, "div._aaqg._aaqh")
    right.click()
    time.sleep(4)
move_next(driver)

### 4.1.6 여러 게시글 정보 수집하기

In [20]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
import re
from bs4 import BeautifulSoup
import unicodedata
from bs4 import BeautifulSoup

driver = webdriver.Chrome()   

# 인스타그램 접속하기
driver.get('https://www.instagram.com/')
time.sleep(2)

# 인스타계정으로 로그인하기
email = '01041303038'   
password = 'moonwalk486!' 

# 인스타그램 로그인 버튼 클릭
driver.find_element(By.CLASS_NAME, '_aa4b ').click()
time.sleep(2)

# 이메일 입력란 찾아서 값 입력
email_input = driver.find_element(By.CLASS_NAME, '_aa4b')
email_input.send_keys(email)

# 비밀번호 입력란 찾아서 값 입력
password_input = driver.find_elements(By.CLASS_NAME, '_aa4b')[1] 
password_input.send_keys(password)

# 로그인 버튼 클릭
login_button = driver.find_element(By.CSS_SELECTOR, 'button[type="submit"]')
login_button.click()
time.sleep(3)  

def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + word
    return url

word = "김채원"
url = insta_searching(word)
driver.get(url)
time.sleep(5)


# 예제 4-3 HTML에서 첫번째 게시글 찾아 클릭하기

# 내용 일부 수정
def select_first(driver):
    first = driver.find_element(By.CSS_SELECTOR, "div._aagw")
    first.click()
    time.sleep(5)
    
select_first(driver)

def get_content(driver):
    # ① 현재 페이지 html 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # ② 본문 내용 가져오기
    try:
        content = soup.select('div._a9zs')[0].text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    
    # ③ 본문 내용에서 해시태그 가져오기(정규식 활용)
    tags = re.findall(r'#[^Ws#,\\]+', content)
   
    # ④ 작성일자 정보 가져오기
    # time class 안에 datetime이라는 항목(년월일시분초 있음)에서 10개 string만 보겠다라는 뜻
    date = soup.select('time._a9ze')[0]['datetime'][:10] 

    # ⑤ 좋아요 수 가져오기
    try:
        like = soup.select('span.html-span')[2].text
    except:
        like = 0
        
    # ⑥ 위치정보 가져오기
    try: 
        place = soup.select('div._aaqm')[0].text
        place = unicodedata.normalize('NFC', place)      
    except:
        place = ''
    # ⑦ 수집한 정보 저장하기
    data = [content, date, like, place, tags]
    return data

# 예제 4-5 다음 게시글 열기
def move_next(driver):
    right = driver.find_element(By.CSS_SELECTOR, "div._aaqg._aaqh")
    right.click()
    time.sleep(4)

# ⑤ 비어있는 변수(results)만들기
results = [ ]


# ⑥→⑦→⑧ 여러 게시물 수집하기
target = 5      # 크롤링할 게시글 수
for i in range(target):
    # 게시글 수집에 오류 발생시(네트워크 문제 등의 이유로)  2초 대기 후, 다음 게시글로 넘어가도록 try, except 구문 활용
    try:
        data = get_content(driver)    # 게시글 정보 가져오기
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2)
        move_next(driver)
    
print(results[:2])


[['03/03 채원이 인스타 게시물❤️너무 이쁘다아….🫢❤️#kimchaewon #chaewon #lesserafim #fearnot #김채원 #채원 #러블리더 #아기치타 #채채 #푸푸 #쌈무 #쌈아치 #쌈무요정 #모닝채원 #김도독 #핌둥이 #피어나 #르세라핌 #キムチェウォン #ルセラフィム', '2024-03-03', '1247', '', ['#kimchaewon ', '#chaewon ', '#le', '#fearnot ', '#김채원 ', '#채원 ', '#러블리더 ', '#아기치타 ', '#채채 ', '#푸푸 ', '#쌈무 ', '#쌈아치 ', '#쌈무요정 ', '#모닝채원 ', '#김도독 ', '#핌둥이 ', '#피어나 ', '#르세라핌 ', '#キムチェウォン ', '#ルセラフィム']], ['02/23 Weverse 채원이💛피.어.나선.물🧸이런 선물…너무 좋아😍#kimchaewon #chaewon #lesserafim #fearnot #김채원 #채원 #러블리더 #아기치타 #채채 #푸푸 #쌈무 #쌈무요정 #쌈아치 #모닝채원 #김도독 #핌둥이 #피어나 #르세라핌 #キムチェウォン #ルセラフィム', '2024-02-23', '5916', '', ['#kimchaewon ', '#chaewon ', '#le', '#fearnot ', '#김채원 ', '#채원 ', '#러블리더 ', '#아기치타 ', '#채채 ', '#푸푸 ', '#쌈무 ', '#쌈무요정 ', '#쌈아치 ', '#모닝채원 ', '#김도독 ', '#핌둥이 ', '#피어나 ', '#르세라핌 ', '#キムチェウォン ', '#ルセラフィム']]]


### 4.1.8 여러 엑셀 파일의 중복을 제거한 후 통합 저장


In [21]:
# 예제 4-7 크롤링 결과 엑셀 저장

import pandas as pd

results_df = pd.DataFrame(results)
results_df = pd.DataFrame(results, columns=['content', 'date', 'like', 'place', 'tags'])
results_df.to_excel('./files/{}.xlsx'.format(word), index=False)


In [22]:
# 여러 개의 저장 파일을 통합
import pandas as pd

jeju_insta_df = pd.DataFrame([])

folder = './files/'
f_list = ['이거.xlsx', '저거.xlsx','김채원.xlsx']

for fname in f_list:
    fpath = folder + fname
    temp = pd.read_excel(fpath)
    jeju_insta_df = pd.concat([jeju_insta_df, temp], ignore_index=True)

jeju_insta_df.columns = ['content', 'date', 'like', 'place', 'tags']

# 예제 4-9 중복 데이터 제거하고 저장하기
jeju_insta_df.drop_duplicates(subset = ["content"] , inplace = True)
jeju_insta_df.to_excel('./files/최종.xlsx', index = False)